In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import random
import os
import glob
import shutil
import itertools
import warnings
from sklearn.metrics import confusion_matrix

In [2]:
os.getcwdb ()

F:\Software\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  """Entry point for launching an IPython kernel.


b'C:\\Users\\Faramarz\\Desktop\\Keras Practice'

# Making files in my directory

In [3]:
os.chdir('cat-dog')
if os.path.isdir('train\dog') is False:
    os.makedirs('train\dog')
    os.makedirs('train\cat')
    os.makedirs('valid\dog')
    os.makedirs('valid\cat')
    os.makedirs('test\dog')
    os.makedirs('test\cat')
    for c in random.sample(glob.glob('dog*'),500):
        shutil.move(c,'train\dog')
    for c in random.sample(glob.glob('cat*'),500):
        shutil.move(c,'train\cat')
    for c in random.sample(glob.glob('dog*'),100):
        shutil.move(c,'valid\dog')
    for c in random.sample(glob.glob('cat*'),100):
        shutil.move(c,'valid\cat')
    for c in random.sample(glob.glob('dog*'),50):
        shutil.move(c,'test\dog')
    for c in random.sample(glob.glob('cat*'),50):
        shutil.move(c,'test\cat')
os.chdir('C:\\Users\\Faramarz\\Desktop\\Keras Practice')

In [4]:
train_path='C:\\Users\\Faramarz\\Desktop\\Keras Practice\\cat-dog\\train'
valid_path='C:\\Users\\Faramarz\\Desktop\\Keras Practice\\cat-dog\\valid'
test_path='C:\\Users\\Faramarz\\Desktop\\Keras Practice\\cat-dog\\test'

In [5]:
train_batch=keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat','dog'],batch_size=10)
valid_batch=keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat','dog'],batch_size=10)
test_batch=keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat','dog'],batch_size=10)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [6]:
assert train_batch.n==1000
assert valid_batch.n==200
assert test_batch.n==100
assert train_batch.num_classes==valid_batch.num_classes==test_batch.num_classes==2

In [ ]:
img,labels=next(train_batch)

In [ ]:
def plot_images(image):
    fig,axis=plt.subplots(1,10, figsize=(20,20))
    axis=axis.flatten()
    for img,ax in zip(image,axis):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
#plot_images(img)

In [ ]:
#print(labels)

In [7]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers import Activation, Dense, Flatten , Conv2D, MaxPool2D

# Pridict

In [8]:
def plot_confusion(cm,classes,normilize=False,title='Confusion Matrix', cmap=plt.cm.Accent):
    plt.imshow(cm, interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes, rotation=45)
    plt.yticks(tick_marks,classes)
    
    if normilize:
        cm=cm.astype('float')/cm.sum(axis=1)[: , np.newaxis]
        print('Normilized Confusion Matrix')
    else:
        print('Without Normalization')
        
    print(cm)
    thresh=cm.max()/2.
    for j,i in itertools.product(range (cm.shape[0]), range(cm.shape[1])):
        plt.text(i,j,cm[i,j],
                color='white' if cm[i,j]>thresh else "black")
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted label')

In [14]:
from sklearn.metrics import plot_confusion_matrix
from keras.preprocessing import image

In [10]:
mobil=keras.applications.mobilenet.MobileNet()

In [27]:
def prepare_image(file):
    img_path="prepare_image/"
    img=image.load_img(img_path+file, target_size=(224,224))
    img_array=image.img_to_array(img)
    img_array_expanded=np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded)

In [42]:
from IPython.display import Image
my_img=Image(filename="prepare_image/"+'1.jpg',width=300,height=200)


In [43]:
preprocessed_img=prepare_image("1.jpg")

In [44]:
prediction= mobil.predict(preprocessed_img)

In [46]:
from keras.applications import imagenet_utils

In [48]:
result=imagenet_utils.decode_predictions(prediction)

In [49]:
result

[[('n02099601', 'golden_retriever', 0.5680741),
  ('n02101388', 'Brittany_spaniel', 0.27741382),
  ('n02099267', 'flat-coated_retriever', 0.08685526),
  ('n04409515', 'tennis_ball', 0.03451738),
  ('n02100583', 'vizsla', 0.010079648)]]

# Converting to Sequential model